In [1]:
# generate moy for phone and invoice look up table 
# it should be updated when a new store/region is added and refresh annually
# moy stands for month of year, it's the percentage of the total volume actuals per month / total volume actuals per year 
# over the past two full calendar years
# this file also include the median monthly volume acutals for the same time period
# eg. if it's 2020-06, moy is calculated from values in 2017 and 2018


# to run this file, three files are needed:
# invoice_actual
# phone_actual


In [87]:
# import libraries
import pandas as pd

In [88]:
# get moy

def get_historic_moy_values(raw_invoice_df,raw_phone_df,start_date = '2017-01-01', end_date = '2020-06-30'):
    
    # the start date should be the first day you would like this moy value
    # the end date shuold be the last day you would like this moy value
    # they are both in str format, e.g."2020-03-01"
    
    
    # obtain all store code
    store_code = raw_invoice_df.store_code.append(raw_phone_df.store_code)
    store_code = pd.DataFrame(store_code.unique(),columns = ['store_code']).dropna()

    
    # create full calendar with store_code
    full_calendar = pd.merge(
        store_code.assign(key=1),
        pd.DataFrame(pd.date_range(start=start_date, end=end_date), columns=["date"]).assign(key=1),
        how='left').drop('key',1)

    # get year, week, month for the full calendar
    full_calendar['year'] = full_calendar['date'].dt.year
    full_calendar['month'] = full_calendar['date'].dt.month
    full_calendar['week'] = full_calendar['date'].dt.week

    # merge with phone and invoice
    # forward fill missing values
    raw_volume_df = pd.merge(full_calendar, raw_invoice_df[['effective_date','store_code','actual']], how='left',
                             left_on=['store_code', 'date'], right_on=['store_code', 'effective_date']).rename(
                             columns={'actual': 'invoice'}).drop('effective_date', 1).fillna(method = 'ffill')

    raw_volume_df = pd.merge(raw_volume_df, raw_phone_df[['effective_date','store_code','actual']], how='left',
                             left_on=['store_code', 'date'], right_on=['store_code', 'effective_date']).rename(
                             columns={'actual': 'phone'}).drop('effective_date', 1).fillna(method = 'ffill')

     # back fill na if there is any
    raw_volume_df = raw_volume_df.fillna(method='bfill')

    # get annually, monthly, and daily median for each store (in all previous years)
    raw_volume_df = raw_volume_df.loc[raw_volume_df['year'] < (max(raw_volume_df['year'].unique()) - 1)]

    # annual sum
    total_annually_df = raw_volume_df.groupby(['store_code']).sum().reset_index().rename(
        columns={'invoice': 'total_annually_invoice', 'phone': 'total_annually_phone'})
    # monthly sum
    total_monthly_df =  raw_volume_df.groupby(['store_code', 'month']).sum().reset_index().rename(
        columns={'invoice': 'total_monthly_invoice', 'phone': 'total_monthly_phone'})
    # monthly median
    average_monthly_df = raw_volume_df.groupby(['store_code', 'month']).median().reset_index().rename(
        columns={'invoice': 'average_monthly_invoice', 'phone': 'average_monthly_phone'})

    # merge back to the original data frame
    raw_volume_df = pd.merge(full_calendar,
                             total_annually_df[['store_code', 'total_annually_invoice', 'total_annually_phone']],
                             how='left',
                             on=['store_code'])
    raw_volume_df = pd.merge(raw_volume_df,
                             total_monthly_df[
                                 ['store_code', 'month', 'total_monthly_invoice', 'total_monthly_phone']],
                             how='left',
                             on=['store_code', 'month'])
    raw_volume_df = pd.merge(raw_volume_df,
                             average_monthly_df[['store_code', 'month', 'average_monthly_invoice', 'average_monthly_phone']],
                             how='left',
                             on=['store_code', 'month'])

    # get moy
    raw_volume_df['invoice_moy_perc'] = raw_volume_df['total_monthly_invoice'] /raw_volume_df[
        'total_annually_invoice']
    raw_volume_df['phone_moy_perc'] = raw_volume_df['total_monthly_phone'] / raw_volume_df[
        'total_annually_phone']


    # get moy file
    moy = raw_volume_df[['date', 'store_code', 'invoice_moy_perc', 'average_monthly_invoice', 'phone_moy_perc',
                         'average_monthly_phone']]

    return moy

In [89]:
# import files
raw_invoice_df = pd.read_csv('invoice_daily_actuals.csv')
raw_phone_df = pd.read_csv('phone_daily_actuals.csv')

# convert to datetitme
raw_invoice_df['effective_date'] = pd.to_datetime(raw_invoice_df['effective_date'], format='%Y%m%d')
raw_phone_df['effective_date'] = pd.to_datetime(raw_phone_df['effective_date'], format="%Y-%m-%d")

# get moy
moy = get_historic_moy_values(raw_invoice_df,raw_phone_df)

# save it
moy.to_csv('moy.csv',header = True, index = False)